<a href="https://colab.research.google.com/github/SilasRu/Oeko3/blob/master/Copy_of_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!git clone https://github.com/SilasRu/Oeko3.git

fatal: destination path 'Oeko3' already exists and is not an empty directory.


In [0]:
import os
import numpy as np
from keras.preprocessing.image import  ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
import keras.models as km
import keras.layers as kl
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Dropout
from keras.utils import to_categorical

In [0]:
import sys
sys.path.append("[Oeko3\\data\\external\\train\\Train]")
path = os.path.join('Oeko3', 'data', 'external', 'train', 'Train')
persons = os.listdir(path)

In [47]:
train_dic = {}
x_train = []
y_train = []
errors = 0
for i in range(len(persons)):
    tempdir = os.path.join(path, persons[i])
    tempfiles = os.listdir(tempdir)
    tempimglist = []
    for j in range(len(tempfiles)):
        temp_img = load_img(os.path.join(tempdir,tempfiles[j]))
        temp_x = img_to_array(temp_img)/255.
        tempimglist.append(temp_x)
        if i ==0 and j == 0:
            temp_x = temp_x.reshape((1,)+temp_x.shape)
            x_train = temp_x
            y_train.append(i)
        else:
            try:
                temp_x = temp_x.reshape((1,)+temp_x.shape)
                print(str(persons[i])+'__'+str(tempfiles[j])+', shape:  '+str(temp_x.shape))
                x_train = np.concatenate((x_train,temp_x),  axis = 0)
                y_train.append(i)
            except:
                errors += 1
                print(str(persons[i])+'__'+str(tempfiles[j]+'   INVALID SHAPE. CAN NOT ADD TO TRAINING SET'))
    train_dic[persons[i]] = np.array(tempimglist)

print(str(errors)+' ERRORS THAT CAN NOT BE ADDED TO TRAINING SET')

spectrograms_berset__1_august_113.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_073.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_012.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_139.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_043.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_001.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_032.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_108.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_002.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_024.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_026.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_079.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_005.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_137.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_086.png, shape:  (1, 480, 640, 3)
spectrograms_berset__1_august_080.png, s

In [42]:
X_train = x_train
y_lab = np.array(y_train)

y_train = to_categorical(y_train)

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64,strides=3, kernel_size=3, activation="relu",
                 input_shape= X_train[0].shape))

#model.add(Conv2D(32, (5, 5), padding='valid', input_shape=(size, size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2 2))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, kernel_initializer='he_normal', bias_initializer='zeros'))
model.add(Activation('tanh'))

    # Softmax??
model.add(Dense(len(persons), kernel_initializer='he_normal', bias_initializer='zeros'))
model.add(Activation('softmax'))

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

SyntaxError: ignored

In [0]:
model.fit(X_train, y_train, validation_split = 0.2, epochs=10)

Train on 184 samples, validate on 47 samples
Epoch 1/10
184/184 [==============================] - 1s 8ms/step - loss: 1.3076 - acc: 0.7446 - val_loss: 11.6556 - val_acc: 0.0000e+00
Epoch 2/10
184/184 [==============================] - 1s 5ms/step - loss: 1.1611 - acc: 0.8750 - val_loss: 2.2624 - val_acc: 0.0000e+00
Epoch 3/10
184/184 [==============================] - 1s 4ms/step - loss: 0.5249 - acc: 0.7446 - val_loss: 2.8671 - val_acc: 0.0000e+00
Epoch 4/10
184/184 [==============================] - 1s 4ms/step - loss: 0.5017 - acc: 0.8750 - val_loss: 3.3976 - val_acc: 0.0000e+00
Epoch 5/10
184/184 [==============================] - 1s 4ms/step - loss: 0.3849 - acc: 0.8750 - val_loss: 1.2924 - val_acc: 0.0000e+00
Epoch 6/10
184/184 [==============================] - 1s 4ms/step - loss: 0.4014 - acc: 0.8750 - val_loss: 2.0900 - val_acc: 0.0000e+00
Epoch 7/10
184/184 [==============================] - 1s 4ms/step - loss: 0.3769 - acc: 0.8750 - val_loss: 2.4052 - val_acc: 0.0000e+00
Ep